In [1]:
# 필요한 라이브러리 설치
!pip install fastapi uvicorn pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.6 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata
os.environ["NGROK_AUTH_TOKEN"] = userdata.get('NGROK_AUTH_TOKEN')

In [24]:
# simple_memory_api.py
from fastapi import FastAPI, Request
from datetime import datetime
import uvicorn
import nest_asyncio


# nest_asyncio 적용 (Jupyter/Colab에서 기존 이벤트 루프에 중첩 이벤트 루프 허용)
nest_asyncio.apply()

# FastAPI 앱 생성
app = FastAPI(title="Memory API")

# 메모리 데이터를 저장할 전역 변수
memory_data = {}

# 사용자 메모리 조회 (모든 메모리 항목을 한번에 반환)
@app.get("/memory/{user_id}")
async def get_user_memory(user_id: str):
    if user_id not in memory_data:
        memory_data[user_id] = {}
    return memory_data[user_id]

# 메모리 저장 (text/plain 형식으로 데이터 받기)
@app.post("/memory/{user_id}")
async def save_memory(user_id: str, request: Request):
    # 사용자 데이터가 없으면 새로 생성
    if user_id not in memory_data:
        memory_data[user_id] = {}

    # 요청 본문을 문자열로 읽기
    text_data = await request.body()
    value = text_data.decode("utf-8")

    # 현재 시간을 키로 사용하여 메모리에 저장
    key = datetime.now().isoformat()

    memory_data[user_id][key] = value

    return {"message": "정보가 저장되었습니다", "data": memory_data[user_id][key]}

# 전체 메모리 삭제
@app.delete("/memory/{user_id}")
async def clear_memory(user_id: str):
    if user_id in memory_data:
        deleted_count = len(memory_data[user_id])
        memory_data[user_id] = {}
        return {"message": f"{deleted_count}개의 메모리 항목이 삭제되었습니다"}
    return {"message": "삭제할 메모리가 없습니다"}

# 메모리 검색 (값에 특정 문자열이 포함된 항목 찾기)
@app.get("/memory/{user_id}/search/{query}")
async def search_memory(user_id: str, query: str):
    if user_id not in memory_data:
        return {"results": []}

    results = []
    for key, data in memory_data[user_id].items():
        if query.lower() in data.lower():
            results.append({
                key: data
            })

    return {"results": results}

In [27]:
from pyngrok import ngrok

# ngrok을 사용하여 외부에서 접근 가능한 URL 생성
def start_server_with_ngrok():
    # ngrok 인증 설정 (ngrok 계정이 있는 경우)
    ngrok.set_auth_token(os.environ["NGROK_AUTH_TOKEN"])

    # ngrok 터널 생성 (Colab에서 사용할 포트 지정)
    port = 8004
    ngrok_tunnel = ngrok.connect(port)
    public_url = ngrok_tunnel.public_url
    print(f"ngrok 터널이 생성되었습니다: {public_url}")
    print(f"API 문서는 {public_url}/docs 에서 확인할 수 있습니다")

    # FastAPI 서버 시작 (non-blocking)
    config = uvicorn.Config(app, host="0.0.0.0", port=port, log_level="info")
    server = uvicorn.Server(config)
    try:
        # run 대신 serve 메서드 사용
        import asyncio
        asyncio.get_event_loop().run_until_complete(server.serve())
    except:
        # 예외가 발생하면 async 방식으로 실행
        server.run()

In [ ]:
start_server_with_ngrok()

ngrok 터널이 생성되었습니다: https://74e2-35-196-74-124.ngrok-free.app
API 문서는 https://74e2-35-196-74-124.ngrok-free.app/docs 에서 확인할 수 있습니다


INFO:     Started server process [648]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8004 (Press CTRL+C to quit)


INFO:     112.158.109.165:0 - "POST /memory/0 HTTP/1.1" 200 OK
INFO:     112.158.109.165:0 - "POST /memory/0 HTTP/1.1" 200 OK
INFO:     112.158.109.165:0 - "GET /0 HTTP/1.1" 404 Not Found
INFO:     112.158.109.165:0 - "GET /memory/0 HTTP/1.1" 200 OK
INFO:     112.158.109.165:0 - "POST /memory/0 HTTP/1.1" 200 OK
INFO:     112.158.109.165:0 - "POST /memory/0 HTTP/1.1" 200 OK
INFO:     112.158.109.165:0 - "POST /memory/0 HTTP/1.1" 200 OK
INFO:     112.158.109.165:0 - "GET /memory/0 HTTP/1.1" 200 OK
INFO:     112.158.109.165:0 - "GET /memory/0 HTTP/1.1" 200 OK
INFO:     112.158.109.165:0 - "POST /memory/0 HTTP/1.1" 200 OK
INFO:     112.158.109.165:0 - "POST /memory/0 HTTP/1.1" 200 OK
INFO:     112.158.109.165:0 - "POST /memory/0 HTTP/1.1" 200 OK
INFO:     112.158.109.165:0 - "GET /memory/0 HTTP/1.1" 200 OK
INFO:     112.158.109.165:0 - "GET /memory/0 HTTP/1.1" 200 OK
INFO:     112.158.109.165:0 - "GET /memory/0/search/%EC%83%9D%EC%9D%BC HTTP/1.1" 200 OK
INFO:     112.158.109.165:0 - "GET /